In [1]:
import numpy as np
import math
import sympy as sym
from scipy.integrate import ode
from scipy.io import loadmat
import scipy
import pickle
import filter_functions
import copy
from IPython.core.debugger import Tracer
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
np.set_printoptions(precision=15)
sym.init_printing()

#MSIS: https://github.com/DeepHorizons/Python-NRLMSISE-00
#import time
from nrlmsise_00_header import *
from nrlmsise_00 import *
#SUBROUTINE GTD7D -- d[5] is the "effective total mass density
#for drag" and is the sum of the mass densities of all species
#in this model, INCLUDING anomalous oxygen.

#define constants
r_earth_const = 6378136.3 #meters
omega_const = 7.2921158553e-5
J_2_const = .00108262617385222
J_3_const = -.00000253241051856772
mu_earth = 3.986004415e14 #m^3/s^2


#Drag:
A_const = 0.9551567 #meters^2; cross-sectional area of satellite
m_const = 10 #kg; mass of satellite
C_D_const = 2.0
theta_dot_const = 7.2921158553e-5 #rad/sec

In [2]:


meas_type = 3

if meas_type == 1:
    meas_file = open('Data Files/meas_range_rangeRate.pkl', 'rb')
elif meas_type == 2:
    meas_file = open('Data Files/meas_az_el.pkl', 'rb')
elif meas_type == 3:
    meas_file = open('Data Files/meas_az_el_range.pkl', 'rb')
    

if meas_type == 3:
    num_of_meas = 3
else:
    num_of_meas = 2

    
#Date of Simulation Details:
#June 24th, 2017 at 6am (this is the date & time at the beginning of the simulation/orbit)
year_init = 2017
month_init = 6
day_init = 24
hour_init = 6
boulder_UT_offset = 6 #Boulder time + 6 hours = UT time
hour_init_UT = hour_init + boulder_UT_offset
    
    

#Canbera Station (DSS 34)
lat_dss34 = math.radians(-35.398333)
lon_dss34 = math.radians(148.981944)
alt_dss34 = 691.75 #m

r_ecef_dss34 = filter_functions.topo2ecef(lat_dss34, lon_dss34, alt_dss34, r_earth_const)
#print(r_ecef_dss34)

#Madrid Station (DSS 65) -- correct position of Madrid Station
lat_dss65 = math.radians(40.427222)
lon_dss65 = math.radians(355.749444)
alt_dss65 = 834.539 #m

r_ecef_dss65 = filter_functions.topo2ecef(lat_dss65, lon_dss65, alt_dss65, r_earth_const)
#print(r_ecef_dss65)

#Goldstone Station (DSS 13) 
lat_dss13 = math.radians(35.247164)
lon_dss13 = math.radians(243.205)
alt_dss13 = 1071.14904 #m

r_ecef_dss13 = filter_functions.topo2ecef(lat_dss13, lon_dss13, alt_dss13, r_earth_const)
#print(r_ecef_dss13)


# read python dict containing measurements
mydict2 = pickle.load(meas_file)
meas_file.close()
measurement_array = mydict2['measurement_array']
truth_xyz = mydict2['truth_pos_vel']
print(np.shape(measurement_array))


(441, 5)


In [3]:

#Propogate reference trajectory and S.T.M.
def orbitpropogator_UKF(t, sigma_pt):
    

    density = calc_MSIS_density(t, sigma_pt)

    #find X acceleration via the F(X) lambdified equation
    state_acc = X_dot_sol_fcn(sigma_pt[0], sigma_pt[1], sigma_pt[2], \
                        sigma_pt[3], sigma_pt[4], sigma_pt[5], density).reshape(6,1)

        
    dx = state_acc.flatten()
    return dx




def calc_MSIS_density(t, X_vector):
    
    state = X_vector[0:3] 
    
    
    day_of_year = math.floor(t/86400) + day_init
    
    #Calculations for theta_gmst (rotation btwn ECI & ECEF for the current time of simulation/orbit)--------------
    hour = hour_init_UT + t/(60*60) #hours (float) since midnight UT
    jd = filter_functions.calc_julian_date(year_init, month_init, day_init, hour) #date of interest in UT

    T_UT = (jd - 2451545)/36525 #calc T_UT at this delta_t/date 
    #calculate theta gmst @ 0 hr using the T_UT and the eq. from Vallado 
    theta_gmst_0_hr = math.radians(100.4606184 + 36000.77005361*T_UT + \
                                           .00038793*T_UT**2 - 2.6e-8*T_UT**3)
    seconds = hour * 60 * 60 #seconds since midnight UT 
    #calculate theta gmst using theta gmst @ 0 hr and seconds 
    #since midnight*the rotation rate of earth
    theta_gmst = theta_gmst_0_hr + omega_const * seconds 
    theta_gmst = theta_gmst % (2*math.pi)


    #calculate position in ECEF & then the lat/lon/alt-----------------------------------------------------------
    r_ecef = filter_functions.eci2ecef(state, theta_gmst)
    (latitude, longitude, altitude) = filter_functions.ecef2geo_lat_lon_alt(r_ecef, r_earth_const)
    
    
    if longitude < 0:
            longitude = longitude + 2*math.pi
    lst = calc_LST(hour, longitude) #lst in units of radians
    
    lon = 0 #dependent only on lst


    Output = nrlmsise_output()
    Input = nrlmsise_input()
    flags = nrlmsise_flags()
    aph = ap_array()

    for i in range(7):
        aph.a[i]=100
    flags.switches[0] = 1
    for i in range(1, 24):
        flags.switches[i]=1

    Input.doy = day_of_year
    Input.year = 0 #/* without effect */
    Input.sec = t
    Input.alt = altitude/1e3 #convert to km
    Input.g_lat = math.degrees(latitude)
    Input.g_long = math.degrees(lon)
    Input.lst = lst
    Input.f107A = 80 #I believe this is a "nominal" value
    Input.f107 = 80
    Input.ap = 4 

    gtd7d(Input, flags, Output)

    density = Output.d[5] #total mass density (grams/m^3, m^3 b/c switches[0] = 1)
    
    return density


#t_UT : current universal time
#current longitude of space object
def calc_LST(t_UT, longitude):
    
    #calculate change in longitude from UT and convert to change in hours/time
    #note: negative lon/lon above 180 is a negative difference in time & opposite for less than 180
    delta_lon = 360 - np.degrees(longitude)
    delta_lst = (delta_lon/360) * 24 #24 hours, unit = hours
    
    t_lst = t_UT - delta_lst 
    if t_lst < 0:
        t_lst = t_lst + 24
    
    #return now if want in hours
    
    #convert to degrees
    t_lst = (t_lst/24) * np.radians(360) #radians
    
    return t_lst

In [4]:
#two body motion force
# **Setup force equations/acceleration/U

#Force equations with J_2
x, y, z, J_2, r_earth, mu, r, J_3 = sym.symbols('x y z J_2 r_earth mu r J_3')


two_body_J2_string = 'mu/r * ( 1 - J_2*(r_earth/r)**2 * (3/2 * (z/r)**2 - 1/2) )'
two_body_J2 = sym.sympify(two_body_J2_string)
two_body_J2 = two_body_J2.subs([(r, sym.sqrt(x**2+y**2+z**2))])
two_body_J2_acc_x = two_body_J2.diff(x)
two_body_J2_acc_y = two_body_J2.diff(y)
two_body_J2_acc_z = two_body_J2.diff(z)


two_body_J2_acc_x = two_body_J2_acc_x.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                              (J_2, J_2_const)])
two_body_J2_acc_y = two_body_J2_acc_y.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                              (J_2, J_2_const)])
two_body_J2_acc_z = two_body_J2_acc_z.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                              (J_2, J_2_const)])
#print('2 body & J2', two_body_J2_acc_x)

x_acc = two_body_J2_acc_x
y_acc = two_body_J2_acc_y
z_acc = two_body_J2_acc_z


#Add drag to J_2 force equations

x_dot, y_dot, z_dot, x_double_dot, y_double_dot, z_double_dot = \
    sym.symbols('x_dot, y_dot, z_dot, x_double_dot, y_double_dot, z_double_dot')
    
C_D, A, m, density, theta_dot, val, val_dot = \
    sym.symbols('C_D A m density theta_dot val, val_dot')

drag_str = ('-(1/2)*C_D*(A/m)*density*'
                'sqrt((x_dot+theta_dot*y)**2 + (y_dot-theta_dot*x)**2 +'
                'z_dot**2)*(val_dot+theta_dot*val)')
drag_symp = sym.sympify(drag_str)

drag_symp = drag_symp.subs([(A, A_const), (m, m_const), (C_D, C_D_const),\
                        (theta_dot, theta_dot_const)])


x_drag_symp = drag_symp.subs([(r, sym.sqrt(x**2+y**2+z**2)), (val, y), (val_dot, x_dot)])
x_acc = x_acc + x_drag_symp

y_drag_symp = drag_symp.subs([(r, sym.sqrt(x**2+y**2+z**2)), (val, x), (val_dot, y_dot)])
y_acc = y_acc + y_drag_symp

z_drag_symp = drag_symp.subs([(r, sym.sqrt(x**2+y**2+z**2)), (val, z), (val_dot, z_dot)])
z_acc = z_acc + z_drag_symp
    


x_acc_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), x_acc)
y_acc_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), y_acc)
z_acc_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), z_acc)

In [5]:


if (meas_type == 1) or (meas_type == 3):
    
    x_s, y_s, z_s, x_sf, y_sf, z_sf, theta, theta_dot, t, x_dot, y_dot, z_dot = \
        sym.symbols('x_s, y_s, z_s, x_sf, y_sf, z_sf, theta, theta_dot, t, x_dot, y_dot, z_dot')

    #define symbolic rho equation
    rho = ('sqrt((x - x_s)**2 + (y - y_s)**2 + (z - z_s)**2)')
    rho = sym.sympify(rho)
    #sub rotation equation of ecef for eci
    rho = rho.subs(x_s, x_sf*sym.cos(omega_const*t) - y_sf*sym.sin(omega_const*t))
    rho = rho.subs(y_s, x_sf*sym.sin(omega_const*t) + y_sf*sym.cos(omega_const*t))
    rho = rho.subs(z_s, z_sf)

    #define symbolic rho dot equation
    rho_dot = ('(x*x_dot + y*y_dot + z*z_dot - (x_dot*x_s+y_dot*y_s)*cos(theta) + \
               theta_dot*(x*x_s + y*y_s)*sin(theta) + (x_dot*y_s - y_dot*x_s)*sin(theta) +\
               theta_dot*(x*y_s - y*x_s)*cos(theta) - z_dot*z_s)/ rho')
    rho_dot = sym.sympify(rho_dot)
    #execute substitutions for rho_dot
    rho_dot = rho_dot.subs(x_s, x_sf) 
    rho_dot = rho_dot.subs(y_s, y_sf) 
    rho_dot = rho_dot.subs(z_s, z_sf) 
    rho_dot = rho_dot.subs('rho', rho)
    rho_dot = rho_dot.subs(theta, omega_const*t)    
    rho_dot = rho_dot.subs(theta_dot, omega_const)

    rho_fcn = lambdify(((x, y, z, x_sf, y_sf, z_sf, t)), rho)
    rho_dot_fcn = lambdify(((x, y, z, x_dot, y_dot, z_dot, x_sf, y_sf, z_sf, t)), rho_dot)


if (meas_type == 2) or (meas_type == 3):
    
    #x_sf, etc. is the sensor pos in ecef
    #x, y, z is the satellite eci
    x_s, y_s, z_s, x_sf, y_sf, z_sf, theta, x, y, z, t = \
        sym.symbols('x_s, y_s, z_s, x_sf, y_sf, z_sf, theta, x, y, z, t')
    x_L, y_L, z_L, X_L_norm, x_range, y_range, z_range, lon, lat, \
        x_sat_ecef, y_sat_ecef, z_sat_ecef, sen_ecef_norm, omega  = \
        sym.symbols('x_L, y_L, z_L, X_L_norm, x_range, y_range, z_range, lon, lat, \
        x_sat_ecef, y_sat_ecef, z_sat_ecef, sen_ecef_norm, omega')
        

    #define symbolic rho equation
    azimuth = ('atan2(x_L, y_L)') #step 4
    azimuth = sym.sympify(azimuth)
    
    elevation = ('asin(z_L/X_L_norm)') #step 4
    elevation = sym.sympify(elevation)
    elevation = elevation.subs(X_L_norm, sym.sqrt(x_L**2 + y_L**2 + z_L**2))
    
    #step 3
    azimuth = azimuth.subs([(x_L, -x_range*sym.sin(lon) + y_range*sym.cos(lon)), \
            (y_L, -x_range*sym.sin(lat)*sym.cos(lon) - y_range*sym.sin(lat)*sym.sin(lon) + z_range*sym.cos(lat))])
    elevation = elevation.subs([(x_L, -x_range*sym.sin(lon) + y_range*sym.cos(lon)), \
            (y_L, -x_range*sym.sin(lat)*sym.cos(lon) - y_range*sym.sin(lat)*sym.sin(lon) + z_range*sym.cos(lat)), \
            (z_L, x_range*sym.cos(lat)*sym.cos(lon) + y_range*sym.cos(lat)*sym.sin(lon) + z_range*sym.sin(lat))])
    
    #step 2
    azimuth = azimuth.subs([(x_range, x_sat_ecef - x_sf), (y_range, y_sat_ecef - y_sf), \
            (z_range, z_sat_ecef - z_sf), (lat, sym.asin(z_sf/sen_ecef_norm)), (lon, sym.atan2(y_sf, x_sf))])
    elevation = elevation.subs([(x_range, x_sat_ecef - x_sf), (y_range, y_sat_ecef - y_sf), \
            (z_range, z_sat_ecef - z_sf), (lat, sym.asin(z_sf/sen_ecef_norm)), (lon, sym.atan2(y_sf, x_sf))])
    
    #step 1
    azimuth = azimuth.subs([(x_sat_ecef, x*sym.cos(theta) + y*sym.sin(theta)), \
                        (y_sat_ecef, -x*sym.sin(theta) + y*sym.cos(theta)), (z_sat_ecef, z), \
                        (sen_ecef_norm, sym.sqrt(x_sf**2 + y_sf**2 + z_sf**2))])
    elevation = elevation.subs([(x_sat_ecef, x*sym.cos(theta) + y*sym.sin(theta)), \
                        (y_sat_ecef, -x*sym.sin(theta) + y*sym.cos(theta)), (z_sat_ecef, z), \
                        (sen_ecef_norm, sym.sqrt(x_sf**2 + y_sf**2 + z_sf**2))])
    
    
    azimuth = azimuth.subs([(theta, omega_const*t), (omega, omega_const)])
    elevation = elevation.subs([(theta, omega_const*t), (omega, omega_const)])
    
    azimuth_fcn = lambdify(((x, y, z, x_sf, y_sf, z_sf, t)), azimuth)
    elevation_fcn = lambdify(((x, y, z, x_sf, y_sf, z_sf, t)), elevation)
    

In [6]:
#State and A matrix


#define the symbolic state matrix
X = sym.Matrix([x, y, z, x_dot, y_dot, z_dot])
X_dot = sym.Matrix([x_dot, y_dot, z_dot, x_acc, y_acc, z_acc])
    

#partial of the force model (x dot) WRT the state vector
A_mat = X_dot.jacobian(X)
#print(A_mat)

A_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), A_mat)
#print(A_sol_fcn(1,2,3,4,5,6,7,8))

#print(X_dot)
X_dot_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), X_dot)

In [7]:
#define observation model (G) and H_tilde


if meas_type == 1:
    G = sym.Matrix([rho, rho_dot])
    
    
elif meas_type == 2:
    G = sym.Matrix([azimuth, elevation])
    
elif meas_type == 3:
    G = sym.Matrix([azimuth, elevation, rho])

#print(G)
G_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, x_sf, y_sf, z_sf, t), G)

#partial derivitive of observation model WRT the state vector
H_tilde = G.jacobian(X)
#print(H_tilde)
H_tilde_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, x_sf, y_sf, z_sf, t), H_tilde)

## Unscented Kalman Filter

In [8]:
#initilaizations

alpha = 5e-4
beta = 2

L = 6 #number of states
k = 3-L
lambdaa = alpha**2*(L + k) - L
gamma = math.sqrt(L+lambdaa)#alpha*math.sqrt(3)


#weights
weights_m_0 = lambdaa/(L+lambdaa)
weights_c_0 = lambdaa/(L+lambdaa) + (1-alpha**2+beta)

weights_m_1 = 1/(2*(L+lambdaa))
weights_c_1 = 1/(2*(L+lambdaa))

pos_perturbation = 100
vel_perturbation = .1

#state 
X_ref = np.array([truth_xyz[0,0] + pos_perturbation, truth_xyz[0,1] + pos_perturbation, truth_xyz[0,2] \
            + pos_perturbation, truth_xyz[0,3] + vel_perturbation, truth_xyz[0,4] + vel_perturbation, \
            truth_xyz[0,5] + vel_perturbation]).reshape(L,1)


x_bar_0 = np.zeros((6,1))

P_sigma_pos = 100
P_sigma_vel = .1
P_bar_0 = np.diag([P_sigma_pos**2, P_sigma_pos**2, P_sigma_pos**2, \
                   P_sigma_vel**2, P_sigma_vel**2, P_sigma_vel**2])

                  
#Define Measurement Noise
if meas_type == 1:
    sigma_rho = .1 
    sigma_rho_dot = .01
    W = np.array([[1/sigma_rho**2, 0], [0, 1/sigma_rho_dot**2]])
    R = np.linalg.inv(W)
    
elif meas_type == 2:
    arcsec_per_rad = 206264.806247 #arcsec/rad
    noise_rad = (1/arcsec_per_rad)  * 5  #5 arcsec noise (suggested by Moriba for a telescope)
    sigma_az = noise_rad
    sigma_el = noise_rad
    W = np.array([[1/sigma_az**2, 0], [0, 1/sigma_el**2]])
    R = np.linalg.inv(W)
    
elif meas_type == 3:
    arcsec_per_rad = 206264.806247 #arcsec/rad
    noise_rad = (1/arcsec_per_rad)  * 5  #5 arcsec noise (suggested by Moriba for a telescope)
    sigma_az = noise_rad
    sigma_el = noise_rad
    sigma_rho = .1 
    W = np.array([[1/sigma_az**2, 0, 0], [0, 1/sigma_el**2, 0], [0, 0, 1/sigma_rho**2]])
    R = np.linalg.inv(W)
    


In [9]:

def compute_sigma_points(sigma_point, P_bar, gamma, L):
    sigma_point = sigma_point.reshape(6,1)
    center_x_hat = sigma_point
    x_hat_tiled = np.tile(sigma_point, L)
    right_x_hat = x_hat_tiled + gamma*scipy.linalg.sqrtm(P_bar)
    left_x_hat = x_hat_tiled - gamma*scipy.linalg.sqrtm(P_bar)
    
    sigma_points = np.array([])

    #build sigma point (comprised of matrices built above)
    sigma_points = np.append(sigma_points, center_x_hat)
    sigma_points = np.append(sigma_points, right_x_hat.T)
    sigma_points = np.append(sigma_points, left_x_hat.T)
    
    return sigma_points

sigma_points_0 = compute_sigma_points(X_ref, P_bar_0, gamma, L)


def calc_sigma_pt_TU(weight_0, weight_1, sigma_points, L):
    
    num_of_pts = 2*L+1
    sigma_points = sigma_points.reshape(num_of_pts, L).T
    #print(sigma_points[:, 0])
    sigma_pt = np.zeros((L, 1))

    for i in range(num_of_pts):
        if i == 0:
            weighted_pt = weight_0*sigma_points[:, i].reshape(6,1)
        else:
            weighted_pt = weight_1*sigma_points[:, i].reshape(6,1)
        
        sigma_pt = sigma_pt + weighted_pt
    
    return sigma_pt



def calc_P_bar_TU(weight_0, weight_1, sigma_pt, sigma_points, L):
    
    num_of_pts = 2*L+1
    sigma_points = sigma_points.reshape(num_of_pts, L).T

    P_bar = np.zeros((L, L))
    
    
    for i in range(num_of_pts):

        difference = sigma_points[:, i].reshape(6,1) - sigma_pt.reshape(6, 1)
        
        if i ==0 :
            weighted_covar = weight_0*difference*difference.T
        else:
            weighted_covar = weight_1*difference*difference.T
            
        #print(weighted_covar)
        P_bar = P_bar + weighted_covar

    
    return P_bar

In [10]:

def compute_measurements(sigma_points, X_s, time):
    
    num_of_pts = 2*L+1
    sigma_points = sigma_points.reshape(num_of_pts, L).T
    Y_mat = np.zeros((num_of_meas, num_of_pts))
    
    for i in range(num_of_pts):

        sigma_pt = sigma_points[:, i]
        
        Y = G_sol_fcn(*sigma_pt, *X_s, time)
        Y = Y.reshape(num_of_meas)
        
        if meas_type != 1:
            if Y[0,0] < 0:
                Y[0,0] = Y[0,0] + 2*math.pi
        
        Y_mat[:, i] = Y
    
    return Y_mat

def compute_y_bar(weight_0, weight_1, Y_mat, L):

    num_of_pts = 2*L+1
    y_bar = np.zeros((num_of_meas, 1))
    
    for i in range(num_of_pts):
        if i == 0:
            weighted_Y = weight_0*Y_mat[:, i].reshape(num_of_meas,1)
        else:
            weighted_Y = weight_1*Y_mat[:, i].reshape(num_of_meas,1)
            
        y_bar = y_bar + weighted_Y

    return y_bar
    


def calc_Pyy(weight_0, weight_1, Y_mat, y_bar, L):
    
    num_of_pts = 2*L+1
    Pyy = np.zeros((num_of_meas, num_of_meas))
    
    for i in range(num_of_pts):

        difference = Y_mat[:, i].reshape(num_of_meas,1) - y_bar.reshape(num_of_meas, 1)
        #print(difference)
        
        if i ==0 :
            weighted_pyy = weight_0*np.dot(difference, difference.T)
        else:
            weighted_pyy = weight_1*np.dot(difference, difference.T)
            
        Pyy = Pyy + weighted_pyy

    return Pyy


def calc_Pxy(weight_0, weight_1, Y_mat, y_bar, sigma_pt, sigma_points, L):
    
    num_of_pts = 2*L+1
    Pxy = np.zeros((L, num_of_meas))
    sigma_points = sigma_points.reshape(num_of_pts, L).T
    
    for i in range(num_of_pts):
        
        y_difference = Y_mat[:, i].reshape(num_of_meas,1) - y_bar.reshape(num_of_meas, 1)
        x_difference = sigma_points[:, i].reshape(L,1) - sigma_pt.reshape(L, 1)
        
        if i ==0 :
            weighted_xy = weight_0*np.dot(x_difference, y_difference.T)
        else:
            weighted_xy = weight_1*np.dot(x_difference, y_difference.T)
            
        Pxy = Pxy + weighted_xy

    return Pxy



In [16]:
#Unscented KF

def execute_ukf(obs_data, sigma_points, X_ref, P_bar_0, prob_dimension, snc_sigma, snc_flag, gamma):

    
    #initializations
    P = P_bar_0
    x_hat = X_ref.reshape(prob_dimension,1)

    P_list = np.array([])
    x_hat_array = np.array([])
    Pyy_array = np.array([])
    Pxy_array = np.array([])
    post_fit_list = np.array([])
    
    #Covariance with SNC: Q Matrix
    Q = np.identity(3)*(snc_sigma)**2
    snc_stm = np.zeros((6,3))
    
    
    #begin UKF
    for obsnum in range(len(obs_data)):#len(obs_data)):
        time = obs_data[obsnum, 0]
        t_init = obs_data[obsnum-1, 0]
        
        print(obsnum)
        
        #set the initial values for the propogator:
        y0 = sigma_points.flatten()
        
 
        if (obs_data[obsnum, 0] != obs_data[obsnum-1, 0]) and obsnum != 0:
            
            result = np.zeros((2*L+1, prob_dimension))
            sigma_points = sigma_points.reshape(2*L+1, prob_dimension)

            for ii in range(2*L+1):
                
                y0 = sigma_points[ii, :]
            
                integrator = ode(orbitpropogator_UKF)
                integrator.set_integrator('dopri5', nsteps=1e6, rtol=1e-12, atol=1e-12)

                integrator.set_initial_value(y0, t_init)
                integrator.integrate(time)
                result[ii, :] = np.array(integrator.y)
   
            sigma_points = result.flatten()
        

        #Time Update- only if measurement is at different time than last measurement
        if (obs_data[obsnum, 0] != obs_data[obsnum-1, 0]) and (obsnum != 0):
            sigma_point_TU = calc_sigma_pt_TU(weights_m_0, weights_m_1, sigma_points, L)
            P_bar_TU = calc_P_bar_TU(weights_c_0, weights_c_1, sigma_point_TU, sigma_points, L)

            
            #Use SNC P_bar if delta t is less than 100 seconds---------------------------
            delta_t = obs_data[obsnum, 0] - obs_data[obsnum-1, 0]
            if obsnum == 0:
                delta_t = 0
            if (delta_t < 15 and snc_flag == True):
                #build SNC STM
                idenity_6_3 = np.array([(delta_t/2)*np.eye(3), np.eye(3)]).reshape(6, 3)
                snc_stm = delta_t*idenity_6_3
                #Covariance with SNC: Q Matrix
                Q = np.identity(3)*(snc_sigma)**2
                P_bar_TU = np.dot(snc_stm, np.dot(Q, snc_stm.T)) + P_bar_TU
            else:
                #don't add SNC, so set Q to zeros
                Q = np.zeros((3,3))
            
            #recompute sigma points
            #sigma_points = compute_sigma_points(sigma_point_TU, P_bar_TU, gamma, L) #sqrtm(Pbar_TU) @ 200 = imaginary
            
        else:
            sigma_point_TU = x_hat
            P_bar_TU = P
            

        #determine station coordinates for observation eq.
        if int(obs_data[obsnum, 1]) == 1:
            #print('35')
            X_s = r_ecef_dss34
        if int(obs_data[obsnum, 1]) == 2:
            #print('65')
            X_s = r_ecef_dss65
        if int(obs_data[obsnum, 1]) == 3:
            #print('13')
            X_s = r_ecef_dss13
            #Tracer() ()
            
            
        
        #compute measurements & weighted average
        Y_mat = compute_measurements(sigma_points, X_s, time)
        #print(Y_mat)
        y_bar = compute_y_bar(weights_m_0, weights_m_1, Y_mat, L)
        #print(y_bar)
        

        #Innovation & Cross-Correlation
        Pyy_term = calc_Pyy(weights_c_0, weights_c_1, Y_mat, y_bar, L)
        Pyy = R + Pyy_term
        #print(R)
        #print(Pyy_term)
        Pxy = calc_Pxy(weights_c_0, weights_c_1, Y_mat, y_bar, sigma_point_TU, sigma_points, L)
        #print(Pxy)

        #Measurement Update
        K = np.dot(Pxy, np.linalg.inv(Pyy))
        #print(K)
        Y_observed = obs_data[obsnum, 2:(2+num_of_meas)].reshape(num_of_meas,1)
        x_hat = sigma_point_TU + np.dot(K, (Y_observed - y_bar))
        #print(x_hat)
        #print(Y_observed - y_bar)
        P = P_bar_TU - np.dot(K, np.dot(Pyy, K.T))

        #recompute sigma points
        sigma_points = compute_sigma_points(x_hat, P, gamma, L)
        
        
        Y = G_sol_fcn(*x_hat, *X_s, time)
        if meas_type != 1:
            if Y[0,0] < 0:
                Y[0,0] = Y[0,0] + 2* math.pi
        post_fit_resid = Y_observed - Y

        
        
        x_hat_array = np.append(x_hat_array, x_hat.reshape(prob_dimension,1))
        P_list = np.append(P_list, P)
        Pyy_array = np.append(Pyy_array, Pyy)
        Pxy_array = np.append(Pxy_array, Pxy)
        post_fit_list = np.append(post_fit_list, post_fit_resid)  



    #resize arrays
    n = len(obs_data)
    P_list.shape = (n, prob_dimension, prob_dimension)
    x_hat_array.shape = (n, prob_dimension)
    Pyy_array.shape = (n, num_of_meas, num_of_meas)
    Pxy_array.shape = (n, prob_dimension, num_of_meas)
    post_fit_list.shape = (n, num_of_meas)

    
    return (x_hat_array, P_list, Pyy_array, Pxy_array, post_fit_list)


In [17]:
prob_dimension = 6
snc_sigma = 1e-8
snc_flag = True

(x_hat_array_ukf, P_list_ukf, Pyy_array_ukf, Pxy_array_ukf, post_fit_list_ukf) = \
        execute_ukf(measurement_array, sigma_points_0, X_ref, P_bar_0, prob_dimension, \
                    snc_sigma, snc_flag, gamma)
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [30]:
print(truth_xyz[0, :])
print(X_ref)

[  4.449499494785039e+06   3.888849001652403e+06   3.313311449784491e+06
  -5.788075686515190e+03   3.890032367467731e+03   3.191832727067075e+03]
[[  4.449599494785039e+06]
 [  3.888949001652403e+06]
 [  3.313411449784491e+06]
 [ -5.787975686515189e+03]
 [  3.890132367467731e+03]
 [  3.191932727067075e+03]]


In [15]:


x_range = 5
y_range = 5
z_range = 5


times = measurement_array[:, 0]
stop_index = len(measurement_array)

calc_display_results(post_fit_list_ukf, measurement_array, meas_type, stop_index)

plot_error_covar_xref(P_list_ukf, x_hat_array_ukf, \
                      truth_xyz, x_range, y_range, z_range, measurement_array, times, stop_index)



NameError: name 'post_fit_list_ukf' is not defined

In [14]:
def calc_display_results(post_fit_list, measurement_array, meas_type, stop_index):
    
    
    rms_1 = 'Range ='
    unit_1 = 'meters'
    ylabel_1 = 'Range Residuals (meters)'
    title_1 = 'Range Post-fit Residuals'
    save_fig_1 = 'postfit_range.png'
    rms_2 = 'Range Rate ='
    unit_2 = 'm/s'
    ylabel_2 = 'Range Rate Residuals (m/s)'
    title_2 = 'Range Rate Post-fit Residuals'
    save_fig_2 = 'postfit_rangeRate.png'
    post_fit_list_new = copy.deepcopy(post_fit_list)
         
    if (meas_type == 2) or (meas_type == 3):
        rms_1 = 'Azimuth ='
        unit_1 = 'degrees'
        rms_2 = 'Elevation ='
        unit_2 = 'degrees'
        post_fit_list_new[:, 0] = np.degrees(post_fit_list[:, 0])
        post_fit_list_new[:, 1] = np.degrees(post_fit_list[:, 1])
        ylabel_1 = 'Azimuth Residuals (degrees)'
        title_1 = 'Azimuth Post-fit Residuals'
        save_fig_1 = 'postfit_az.png'
        ylabel_2 = 'Elevation Residuals (degrees)'
        title_2 = 'Elevation Post-fit Residuals'
        save_fig_2 = 'postfit_el_rate.png'

        
    

    times = measurement_array[:stop_index,0]/(60*60)
    
    indices_1 = np.where(measurement_array[:stop_index, 1] == 1)[0]
    indices_2 = np.where(measurement_array[:stop_index, 1] == 2)[0]
    indices_3 = np.where(measurement_array[:stop_index, 1] == 3)[0]

    
    
    #Post-fit
    print('Post-fit RMS:')
    post_fit_1_list_4RMS = post_fit_list_new[:, 0]
    postfit_1_rms = np.sqrt(np.mean(np.square(post_fit_1_list_4RMS)))
    print(rms_1, postfit_1_rms, unit_1)

    post_fit_2_list_4RMS = post_fit_list_new[:, 1]
    postfit_2_rms = np.sqrt(np.mean(np.square(post_fit_2_list_4RMS)))
    print(rms_2, postfit_2_rms, unit_2)
    
    if meas_type == 3:
        post_fit_3_list_4RMS = post_fit_list_new[:, 2]
        postfit_3_rms = np.sqrt(np.mean(np.square(post_fit_3_list_4RMS)))
        print('Range =', postfit_3_rms, 'meters')
    
    

   
    #Post-fit Residuals
    fig = plt.figure()
    plt.scatter(times[indices_1], post_fit_list_new[indices_1, 0], s=70, c='g', marker='x')
    plt.scatter(times[indices_2], post_fit_list_new[indices_2, 0], s=70, c='b', marker='+')
    plt.scatter(times[indices_3], post_fit_list_new[indices_3, 0], s=70, c='m', marker='*')
    #plt.scatter(times, post_fit_list[:, 0])
    plt.ylabel(ylabel_1, fontsize=18)
    plt.xlabel('Time (hours)', fontsize=18)
    plt.title(title_1, fontsize=18)
    legend_names = ['Station 1', 'Station 2', 'Station 3']
    plt.legend(legend_names, fontsize=16)
    #plt.ylim([-.01,.01])
    plt.tight_layout()
    plt.show()
    #fig.savefig(save_fig_1)

    fig = plt.figure()
    plt.scatter(times[indices_1], post_fit_list_new[indices_1, 1], s=70, c='g', marker='x')
    plt.scatter(times[indices_2], post_fit_list_new[indices_2, 1], s=70, c='b', marker='+')
    plt.scatter(times[indices_3], post_fit_list_new[indices_3, 1], s=70, c='m', marker='*')
    #plt.scatter(times, post_fit_list[:, 1])
    plt.ylabel(ylabel_2, fontsize=18)
    plt.xlabel('Time (hours)', fontsize=18)
    plt.title(title_2, fontsize=18)
    legend_names = ['Station 1', 'Station 2', 'Station 3']
    plt.legend(legend_names, fontsize=16)
    #plt.ylim([-.01,.01])
    plt.tight_layout()
    plt.show()
    #fig.savefig(save_fig_2)
    
    if meas_type == 3:
        fig = plt.figure()
        plt.scatter(times[indices_1], post_fit_list_new[indices_1, 2], s=70, c='g', marker='x')
        plt.scatter(times[indices_2], post_fit_list_new[indices_2, 2], s=70, c='b', marker='+')
        plt.scatter(times[indices_3], post_fit_list_new[indices_3, 2], s=70, c='m', marker='*')
        #plt.scatter(times, post_fit_list[:, 1])
        plt.ylabel('Range Residuals (meters)', fontsize=18)
        plt.xlabel('Time (days)', fontsize=18)
        plt.title('Range Post-fit Residuals', fontsize=18)
        legend_names = ['Station 1', 'Station 2', 'Station 3']
        plt.legend(legend_names, fontsize=16)
        #plt.ylim([-.01,.01])
        plt.tight_layout()
        plt.show()
        #fig.savefig('postfit_range.png')
    
    
    
def plot_error_covar_xref(P_list, x_ref_updated_list, obs_data_truth, x_range, \
                          y_range, z_range, measurement_array, time, stop_index):
    
    #Compare to the Truth Data : Estimation Errors------

    times = time[:stop_index]/(60*60)
    
    indices_1 = np.where(measurement_array[:stop_index, 1] == 1)[0]
    indices_2 = np.where(measurement_array[:stop_index, 1] == 2)[0]
    indices_3 = np.where(measurement_array[:stop_index, 1] == 3)[0]
    
    
    x_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 0, 0]))*3
    x_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 0, 0]))*3
    x_error = x_ref_updated_list[:,0] - obs_data_truth[:stop_index, 0]
    
    y_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 1, 1]))*3
    y_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 1, 1]))*3
    y_error = x_ref_updated_list[:,1] - obs_data_truth[:stop_index, 1]
    
    z_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 2, 2]))*3
    z_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 2, 2]))*3
    z_error = x_ref_updated_list[:,2] - obs_data_truth[:stop_index, 2]
    
    #error_pos_norm = np.sqrt(x_error**2 + y_error**2 + z_error**2)
    print('Position RMS:')
    error_x_pos_rms_3D = np.sqrt(np.mean(np.square(x_error)))
    print('X =', error_x_pos_rms_3D, 'meters')
    
    error_y_pos_rms_3D = np.sqrt(np.mean(np.square(y_error)))
    print('Y =', error_y_pos_rms_3D, 'meters')
    
    error_z_pos_rms_3D = np.sqrt(np.mean(np.square(z_error)))
    print('Z =', error_z_pos_rms_3D, 'meters')
    
    pos_rms = np.sqrt(error_x_pos_rms_3D**2 + error_y_pos_rms_3D**2 + error_z_pos_rms_3D**2)
    print('Overall =', pos_rms/1e3, 'km')
    

    #x Position
    fig = plt.figure()
    plt.plot(times, x_covar_env_upper, label='_nolegend_')
    plt.plot(times, x_covar_env_lower, label='_nolegend_')
    plt.scatter(times[indices_1], x_error[indices_1], s=70, c='g', marker='x')
    plt.scatter(times[indices_2], x_error[indices_2], s=70, c='b', marker='+')
    plt.scatter(times[indices_3], x_error[indices_3], s=70, c='m', marker='*')
    #plt.scatter(times, x_error)
    plt.ylabel('meters', fontsize=18)
    plt.xlabel('Time (hours)', fontsize=18)
    legend_names = ['Station 1', 'Station 2', 'Station 3']
    plt.legend(legend_names, fontsize=16)
    plt.title('UKF X Position Covariance Envelope', fontsize=18)
    plt.ylim([-x_range,x_range])
    #plt.xlim([0,time_hrs[-1]])
    plt.show()
    #fig.savefig('x_pos_error.png')

    #y Position 
    fig = plt.figure()
    plt.plot(times, y_covar_env_upper, label='_nolegend_')
    plt.plot(times, y_covar_env_lower, label='_nolegend_')
    plt.scatter(times[indices_1], y_error[indices_1], s=70, c='g', marker='x')
    plt.scatter(times[indices_2], y_error[indices_2], s=70, c='b', marker='+')
    plt.scatter(times[indices_3], y_error[indices_3], s=70, c='m', marker='*')
    #plt.scatter(times, y_error)
    plt.ylabel('meters', fontsize=18)
    plt.xlabel('Time (hours)', fontsize=18)
    plt.legend(legend_names, fontsize=16)
    plt.title('UKF Y Position Covariance Envelope', fontsize=18)
    plt.ylim([-y_range,y_range])
    #plt.xlim([0,time_hrs[-1]])
    plt.show()
    #fig.savefig('y_pos_error.png')

    #z Position
    fig = plt.figure()
    plt.plot(times, z_covar_env_upper, label='_nolegend_')
    plt.plot(times, z_covar_env_lower, label='_nolegend_')
    plt.scatter(times[indices_1], z_error[indices_1], s=70, c='g', marker='x')
    plt.scatter(times[indices_2], z_error[indices_2], s=70, c='b', marker='+')
    plt.scatter(times[indices_3], z_error[indices_3], s=70, c='m', marker='*')
    #plt.scatter(times, z_error)
    plt.ylabel('meters', fontsize=18)
    plt.xlabel('Time (hours)', fontsize=18)
    plt.legend(legend_names, fontsize=16)
    plt.title('UKF Z Position Covariance Envelope', fontsize=18)
    plt.ylim([-z_range,z_range])
    #plt.xlim([0,time_hrs[-1]])
    plt.show()
    #fig.savefig('z_pos_error.png')
    
    #x Velocity
    x_dot_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 3, 3]))*3
    x_dot_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 3, 3]))*3
    x_vel_error = x_ref_updated_list[:,3] - obs_data_truth[:stop_index, 3]

    fig = plt.figure()
    plt.plot(times, x_dot_covar_env_upper, label='_nolegend_')
    plt.plot(times, x_dot_covar_env_lower, label='_nolegend_')
    plt.scatter(times[indices_1], x_vel_error[indices_1], s=70, c='g', marker='x')
    plt.scatter(times[indices_2], x_vel_error[indices_2], s=70, c='b', marker='+')
    plt.scatter(times[indices_3], x_vel_error[indices_3], s=70, c='m', marker='*')
    #plt.scatter(times, x_vel_error)
    plt.ylabel('meters/second', fontsize=18)
    plt.xlabel('Time (hours)', fontsize=18)
    plt.legend(legend_names, fontsize=16)
    plt.title('UKF X Velocity Estimation Covariance Envelope', fontsize=18)
    #plt.ylim([-x_range,x_range])
    #plt.xlim([0,time_hrs[-1]])
    plt.show()
    #fig.savefig('x_vel_error.png')

    #y Velocity
    y_dot_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 4, 4]))*3
    y_dot_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 4, 4]))*3
    y_vel_error = x_ref_updated_list[:,4] - obs_data_truth[:stop_index, 4]

    fig = plt.figure()
    plt.plot(times, y_dot_covar_env_upper, label='_nolegend_')
    plt.plot(times, y_dot_covar_env_lower, label='_nolegend_')
    plt.scatter(times[indices_1], y_vel_error[indices_1], s=70, c='g', marker='x')
    plt.scatter(times[indices_2], y_vel_error[indices_2], s=70, c='b', marker='+')
    plt.scatter(times[indices_3], y_vel_error[indices_3], s=70, c='m', marker='*')
    #plt.scatter(times, y_vel_error)
    plt.ylabel('meters/second', fontsize=18)
    plt.xlabel('Time (hours)', fontsize=18)
    plt.legend(legend_names, fontsize=16)
    plt.title('UKF Y Velocity Estimation Covariance Envelope', fontsize=18)
    #plt.ylim([-y_range,y_range])
    #plt.xlim([0,time_hrs[-1]])
    plt.show()
    #fig.savefig('y_vel_error.png')

    #z Velocity
    z_dot_covar_env_upper = np.sqrt(abs(P_list[:stop_index, 5, 5]))*3
    z_dot_covar_env_lower = -np.sqrt(abs(P_list[:stop_index, 5, 5]))*3
    z_vel_error = x_ref_updated_list[:,5] - obs_data_truth[:stop_index, 5]

    fig = plt.figure()
    plt.plot(times, z_dot_covar_env_upper, label='_nolegend_')
    plt.plot(times, z_dot_covar_env_lower, label='_nolegend_')
    plt.scatter(times[indices_1], z_vel_error[indices_1], s=70, c='g', marker='x')
    plt.scatter(times[indices_2], z_vel_error[indices_2], s=70, c='b', marker='+')
    plt.scatter(times[indices_3], z_vel_error[indices_3], s=70, c='m', marker='*')
    #plt.scatter(times, z_vel_error)
    plt.ylabel('meters/second', fontsize=18)
    plt.xlabel('Time (hours)', fontsize=18)
    plt.legend(legend_names, fontsize=16)
    plt.title('UKF Z Velocity Estimation Covariance Envelope', fontsize=18)
    #plt.ylim([-z_range,z_range])
    #plt.xlim([0,time_hrs[-1]])
    plt.show()
    #fig.savefig('z_vel_error.png')
    
    
    print('Velocity RMS:')
    error_x_vel_rms_3D = np.sqrt(np.mean(np.square(x_vel_error)))
    print('X =', error_x_vel_rms_3D, 'meters/second')
    
    error_y_vel_rms_3D = np.sqrt(np.mean(np.square(y_vel_error)))
    print('Y =', error_y_vel_rms_3D, 'meters/second')
    
    error_z_vel_rms_3D = np.sqrt(np.mean(np.square(z_vel_error)))
    print('Z =', error_z_vel_rms_3D, 'meters/second')
    
    vel_rms = np.sqrt(error_x_vel_rms_3D**2 + error_y_vel_rms_3D**2 + error_z_vel_rms_3D**2)
    print('Overall =', vel_rms/1e3, 'km/s')

    
    """
    fig.savefig('x_pos_error.png')
    fig.savefig('y_pos_error.png')
    fig.savefig('z_pos_error.png')
    fig.savefig('x_vel_error.png')
    fig.savefig('y_vel_error.png')
    fig.savefig('z_vel_error.png')
    """